In [ ]:
# import the required libs
import numpy as np
import pylab as pl
import commonUtil
import os
import pandas as pd
import statsmodels.api as sm
import datetime

from scipy.interpolate import spline
from sklearn import linear_model

#To get all query conditions into a dictionary
queryCondition = commonUtil.handle_query_condition()
ISOK_ALL_PARA = queryCondition['ISOK_ALL_PARA']
REPORT_TYPE = queryCondition['REPORT_TYPE']

DB_SET = queryCondition['DB_SET']
START_TIME = queryCondition['START_TIME']
END_TIME = queryCondition['END_TIME']
INTERVAL = queryCondition['INTERVAL']
GENERAL_ERROR = queryCondition['GENERAL_ERROR']
DB_CONN_ID = DB_SET[0]
#To print the error of query conditon, only print once
for err_id in range(len(queryCondition['GENERAL_ERROR'])):
    print queryCondition['GENERAL_ERROR'][err_id]               

In [ ]:
'''
According to original dataframe we can get a serial data set 
and use it to form a new dataframe.
'''
def get_train_df(dataframe_hour):
    try:
        #Get hour from original dataframe
        x_hour_list = list(dataframe_hour['HOURS'].values)
        #Get date from original dataframe
        date_ori_list = list(dataframe_hour['DATE'].values)
        #Get y_max_log_percent from original dataframe
        y_max_log_percent = list(dataframe_hour['ACTIVE_LOG_USED_PERCENT_MAX'].values)
        #Get y_avg_log_percent from original dataframe
        y_avg_log_percent = list(dataframe_hour['ACTIVE_LOG_USED_PERCENT_AVG'].values)
        
        ##Get timestamp using hour and date from original dataframe
        ts_list = []
        for i in range(len(x_hour_list)):
            hr_str = str(x_hour_list[i])
            if len(hr_str) == 1:
                hr_str = '0' + hr_str + ':00:00'
            else:
                hr_str = hr_str + ':00:00'
            dt_str = str(date_ori_list[i]) + ' ' + hr_str
            time_tuple = time.strptime(dt_str, '%Y-%m-%d %H:00:00')
            ts = int(time.mktime(time_tuple))
            ts_list.append(ts)
        '''
        Judge whether data set is serial or not
        '''  
        index_list = []
        tmp_index = []
        for j in range(len(ts_list) - 1):
            if j not in tmp_index:
                tmp_index.append(j)
            if (ts_list[j + 1] - ts_list[j]) == 3600:
                tmp_index.append(j + 1)
                if (j + 1) == len(ts_list) - 1:
                    index_list.append(tmp_index)
                    tmp_index = None
            else:
                index_list.append(tmp_index)
                tmp_index = []
                if (j + 1) == len(ts_list) - 1:
                    tmp_index.append(j + 1)
                    index_list.append(tmp_index)
                    tmp_index = []
        max_continuous_index = []
        
        #for l in range(len(index_list)):
            #max_len = len(index_list[0])
            #if max_len <= len(index_list[l]):
                #max_len = len(index_list[l])
                #max_continuous_index = index_list[l]
        
        #Get the newest continuous data set
        from_end_len = len(index_list) - 1
        while from_end_len >= 0:
            if(len(index_list[from_end_len]) >= 100):
                max_continuous_index = index_list[len(index_list) - 1]
                break
            else:
                if from_end_len == 0:
                    max_continuous_index = index_list[len(index_list) - 1]
                from_end_len_ -= 1
        x_hour_list = x_hour_list[max_continuous_index[0]:max_continuous_index[len(max_continuous_index) - 1] + 1]
        date_ori_list = date_ori_list[max_continuous_index[0]:max_continuous_index[len(max_continuous_index) - 1] + 1]
        y_max_log_percent = y_max_log_percent[max_continuous_index[0]:max_continuous_index[len(max_continuous_index) - 1] + 1]
        y_avg_log_percent = y_avg_log_percent[max_continuous_index[0]:max_continuous_index[len(max_continuous_index) - 1] + 1]
        
        df = pd.DataFrame()#[date_ori_list, x_hour_list, y_max_log_percent, y_avg_log_percent], columns=['DATE', 'HOURS', 'ACTIVE_LOG_USED_PERCENT_MAX', 'ACTIVE_LOG_USED_PERCENT_AVG'])
        df['DATE'] = pd.Series(date_ori_list)
        df['HOURS'] = pd.Series(x_hour_list)
        df['ACTIVE_LOG_USED_PERCENT_MAX'] = pd.Series(y_max_log_percent)
        df['ACTIVE_LOG_USED_PERCENT_AVG'] = pd.Series(y_avg_log_percent)
        return df
    except Exception as ex:
        print str(ex)
#To print the error of query conditon, only print once
for err_id in range(len(GENERAL_ERROR)):
    print GENERAL_ERROR[err_id]
    
#To get data and draw graph by data.
if(ISOK_ALL_PARA and (REPORT_TYPE == 'ALL' or REPORT_TYPE == 'LOG')):
    DB_CONN_ID = DB_SET[0]
    #Get data from the target database to assemble a data frame that will be used for the following graph
    dataframe_interval = %sql SELECT char(date(COLLECTED)) as DATE, \
                              hour(COLLECTED) as HOURS, \
                              decimal(round(avg(decimal(TOTAL_LOG_USED, 17, 2)) / 1024, 2), 17, 2) as TOTAL_LOG_USED_AVG_KB, \
                              decimal(round(max(decimal(TOTAL_LOG_USED, 17, 2)) / 1024, 2), 17, 2) as TOTAL_LOG_USED_MAX_KB, \
                              decimal(round(avg(decimal(TOTAL_LOG_AVAILABLE, 17, 2)) / 1024, 2), 17, 2) as TOTAL_LOG_AVAILABLE_AVG_KB, \
                              decimal(round(max(decimal(TOTAL_LOG_AVAILABLE, 17, 2)) / 1024, 2), 17, 2) as TOTAL_LOG_AVAILABLE_MAX_KB, \
                              decimal(round(avg(PCENTACTIVELOGUSED), 2), 17, 2) as ACTIVE_LOG_USED_PERCENT_AVG, \
                              decimal(round(max(PCENTACTIVELOGUSED), 2), 17, 2) as ACTIVE_LOG_USED_PERCENT_MAX, \
                              decimal(round(avg(decimal(TOT_LOG_USED_TOP, 17, 2)) / 1024, 2), 17, 2) as TOT_LOG_USED_TOP_AVG_KB, \
                              decimal(round(max(decimal(TOT_LOG_USED_TOP, 17, 2)) / 1024, 2), 17, 2) as TOT_LOG_USED_TOP_MAX_KB, \
                              decimal(round(avg(decimal(SEC_LOG_USED_TOP, 17, 2))/1024, 2), 17, 2) as SEC_LOG_USED_TOP_AVG_KB, \
                              decimal(round(max(decimal(SEC_LOG_USED_TOP, 17, 2)) / 1024, 2), 17, 2) as SEC_LOG_USED_TOP_MAX_KB, \
                              avg(SEC_LOGS_ALLOCATED) as SEC_LOGS_ALLOCATED_AVG, max(SEC_LOGS_ALLOCATED) as SEC_LOGS_ALLOCATED_MAX \
                              from IBM_DSM_VIEWS.MON_GET_TRANSACTION_LOG \
                              where dbconn_id='{DB_CONN_ID}' and collected >= '{START_TIME}' and collected < '{END_TIME}' \
                              group by date(COLLECTED), hour(COLLECTED) \
                              order by DATE,HOURS
    
    if dataframe_interval.empty:
        print 'For Log: The query result is empty, please check your query parameters.\n'
        if os.path.exists("log.csv"):
            !rm log.csv
    else:
        #To save log data for generating table
        if os.path.exists("log.csv"):
            !rm log.csv
        dataframe_interval.to_csv("log.csv", index_label = "INDEX")
        
        dataframe_hour_train = %sql SELECT char(date(COLLECTED)) as DATE, \
                                    hour(COLLECTED) as HOURS, \
                                    decimal(round(avg(decimal(TOTAL_LOG_USED, 17, 2)) / 1024, 2), 17, 2) as TOTAL_LOG_USED_AVG_KB, \
                                    decimal(round(max(decimal(TOTAL_LOG_USED, 17, 2)) / 1024, 2), 17, 2) as TOTAL_LOG_USED_MAX_KB, \
                                    decimal(round(avg(decimal(TOTAL_LOG_AVAILABLE, 17, 2)) / 1024, 2), 17, 2) as TOTAL_LOG_AVAILABLE_AVG_KB, \
                                    decimal(round(max(decimal(TOTAL_LOG_AVAILABLE, 17, 2)) / 1024, 2), 17, 2) as TOTAL_LOG_AVAILABLE_MAX_KB, \
                                    decimal(round(avg(PCENTACTIVELOGUSED), 2), 17, 2) as ACTIVE_LOG_USED_PERCENT_AVG, \
                                    decimal(round(max(PCENTACTIVELOGUSED), 2), 17, 2) as ACTIVE_LOG_USED_PERCENT_MAX, \
                                    decimal(round(avg(decimal(TOT_LOG_USED_TOP, 17, 2)) / 1024, 2), 17, 2) as TOT_LOG_USED_TOP_AVG_KB, \
                                    decimal(round(max(decimal(TOT_LOG_USED_TOP, 17, 2)) / 1024, 2), 17, 2) as TOT_LOG_USED_TOP_MAX_KB, \
                                    decimal(round(avg(decimal(SEC_LOG_USED_TOP, 17, 2))/1024, 2), 17, 2) as SEC_LOG_USED_TOP_AVG_KB, \
                                    decimal(round(max(decimal(SEC_LOG_USED_TOP, 17, 2)) / 1024, 2), 17, 2) as SEC_LOG_USED_TOP_MAX_KB, \
                                    avg(SEC_LOGS_ALLOCATED) as SEC_LOGS_ALLOCATED_AVG, \
                                    max(SEC_LOGS_ALLOCATED) as SEC_LOGS_ALLOCATED_MAX \
                                    from IBM_DSM_VIEWS.MON_GET_TRANSACTION_LOG \
                                    where dbconn_id='{DB_CONN_ID}' and collected < '{END_TIME}' \
                                    group by date(COLLECTED), hour(COLLECTED) \
                                    order by DATE,HOURS
        
        #Get continuous data to build a predict model
        df_train = get_train_df(dataframe_hour_train)
        
        #Get the hour data as x-axis from the data frame and convert the data into a numpy array
        x_hour_list_train = list(df_train['HOURS'].values)
        date_ori_train = list(df_train['DATE'].values)
        #Get the data as y-axis from the data frame and convert the data into a numpy array
        y_max_log_percent_list_train = list(df_train['ACTIVE_LOG_USED_PERCENT_MAX'].values)
        #Get the data as y-axis from the data frame and convert the data into a numpy array
        y_avg_log_percent_list_train = list(df_train['ACTIVE_LOG_USED_PERCENT_AVG'].values)
        
        #Predict INTERVAL points
        predict_count = float('%.1f' % INTERVAL)

        ######################
        ## create predictive model with time series algorithm ARIMA(Autoregressive–moving-average model)
        ## For the source data are non-seasonal, ARIMA(p, d, q) instead of ARIMA(p, d, q, s) is used to build model
        ## ARIMA(p, d, q): 
        ##     AR(p) represents the P order autoregressive process; 
        ##     MA(q) represents the moving average process of the Q order; 
        ##     d respresents the number of difference
        ## Refer to notebook ARIMA-Analysis for how to determine number of p, d, q with detail
        
        ## Generate pd.Series object for Time Series analysis 
        df_max = pd.Series(y_max_log_percent_list_train)
        df_avg = pd.Series(y_avg_log_percent_list_train)
        
        datetime_str_train = []
        for id_1 in range(len(date_ori_train)):
            tmp_hour = str(x_hour_list_train[id_1])
            if(len(tmp_hour) == 1):
                tmp_hour = '0' + tmp_hour
            tmp_dt = str(date_ori_train[id_1]) + ' ' + tmp_hour
            datetime_str_train.append(tmp_dt)
        ## 
        df_max.index = pd.PeriodIndex(start = datetime_str_train[0], end = datetime_str_train[len(datetime_str_train) - 1])
        df_avg.index = pd.PeriodIndex(start = datetime_str_train[0], end = datetime_str_train[len(datetime_str_train) - 1])
        
        ## Apply data to ARIMA algorithm to generate Time Series analysis model with forcast data
        model_max = sm.tsa.ARIMA(df_max,(7, 0, 1)).fit()
        model_avg = sm.tsa.ARIMA(df_avg,(7, 0, 1)).fit()
        
        ## Handling interval
        ## interval<=10, shows number of real data and predictive data same to interval
        ## interval>10, shows number of real data same to interval and 10 predictive data
   
        #Define a empty var x_ticks to store x-axis ticks(marks)
        x_ticks = []
        #Define a empty var x_ticks_lables to restore x-axis labels
        x_ticks_lables = []
        #Define a empty var y_ticks to store y-axis ticks(marks)
        y_max_log_percent_list = list(dataframe_interval['ACTIVE_LOG_USED_PERCENT_MAX'].values)
        y_avg_log_percent_list = list(dataframe_interval['ACTIVE_LOG_USED_PERCENT_AVG'].values)
        
        end_dt_str = END_TIME[0:13] + ":00:00"
        end_tuple = time.strptime(end_dt_str, '%Y-%m-%d %H:00:00')
        end_timestamp = time.mktime(end_tuple)
        
        previous_hour = None
        predict_hour = None
        aft_hour_st = None
        pre_hour_st = end_timestamp - 3600.0 * INTERVAL
        tmp_datetime = datetime.datetime.fromtimestamp(pre_hour_st)
        previous_hour = tmp_datetime.strftime("%Y-%m-%d %H:00:00")[0:13]
        
        aft_hour_st = end_timestamp + 3600 * predict_count
            
        tmp_datetime = datetime.datetime.fromtimestamp(aft_hour_st)
        predict_hour = tmp_datetime.strftime("%Y-%m-%d %H:00:00")[0:13]
        
        ## Define format of x-Axis
        date_all = []
        x_hour = []
        loop_count = int((aft_hour_st - pre_hour_st)/3600.0)
        for id_2 in range(loop_count):
            tmp_hour_st = pre_hour_st + id_2 * 3600.0
            tmp_datetime = datetime.datetime.fromtimestamp(tmp_hour_st)
            date_all.append(tmp_datetime.strftime("%Y-%m-%d %H:00:00")[0:10])
            x_hour.append(tmp_datetime.strftime("%Y-%m-%d %H:00:00")[11:13])
        commonUtil.format_x_axis(date_all, x_hour, x_ticks, x_ticks_lables)
        ## Apply data to model to generate forcast data
        predict_array_max = model_max.predict(datetime_str_train[len(datetime_str_train) - 1], predict_hour, dynamic=True)
        #print 'predict_array_max=',predict_array_max
        #predict_array_max_list = list(predict_array_max)
        predict_array_max = predict_array_max[1:INTERVAL + 1]
        ## Prepare data for Y-Axis
        y_max_log_percent_list = y_max_log_percent_list[-INTERVAL:] + list(predict_array_max)
        
        ## Apply data to model to generate forcast data
        predict_array_avg = model_avg.predict(datetime_str_train[len(datetime_str_train) - 1], predict_hour, dynamic=True)
        predict_array_avg =  predict_array_max = predict_array_avg[1:INTERVAL + 1]
        y_avg_log_percent_list = y_avg_log_percent_list[-INTERVAL:] + list(predict_array_avg)
        
        #ax = df_final.ix[previous_ten_hour:].plot(ax=ax)
        #predict_sunspots.plot(ax=ax)
        fig, ax = pl.subplots(figsize=(24, 7))
        figure_title = 'Max and Average Log Usage Percentage by Hour\n'
        pl.title(figure_title, fontsize = 14, fontweight = 'bold')
        x_lable = 'Hours'
        #To set x-axis label
        pl.xlabel(x_lable)
        #To set y-axis label
        pl.ylabel(u'ACTIVE_LOG_USED_PERCENT %')
        #To set grid line style according to your requirement
        pl.grid(True, ls = '--', color = '#2c628b', alpha = 0.05)
        pl.xticks(x_ticks, x_ticks_lables, rotation = 90)
        
        ##To scatter the date of y_max_log_percent_list and mark the data point 
        for id in range(0,len(y_max_log_percent_list),2):
            if(y_max_log_percent_list[id] == 0.0):#If no data,drawing a empty circle
                if id >= len(y_max_log_percent_list) - predict_count and id < len(y_max_log_percent_list):
                    #This is the data of prediction and empty
                    pl.scatter(x_ticks[id], y_max_log_percent_list[id], c = 'r') 
                else:
                    #This is the real data of y_max_log_percent and empty
                    pl.scatter(x_ticks[id], y_max_log_percent_list[id], c = '', marker = 'o', edgecolors = 'r', s = 50)
            else:#Data not empty
                if id >= len(y_max_log_percent_list) - predict_count and id < len(y_max_log_percent_list):
                    #This is the data of prediction and not empty
                    pl.scatter(x_ticks[id], y_max_log_percent_list[id], c = 'r') 
                    pl.text(x_ticks[id], y_max_log_percent_list[id], '%.1f' % y_max_log_percent_list[id], fontsize = 9)
                else:
                    #This is the real data of y_max_log_percent and not empty
                    pl.scatter(x_ticks[id], y_max_log_percent_list[id], c = '#60bdae') 
                    pl.text(x_ticks[id], y_max_log_percent_list[id], '%.1f' % y_max_log_percent_list[id], fontsize = 9)
        
        #### smooth the lines
        for id in range(1, len(y_avg_log_percent_list), 2):
            if(y_avg_log_percent_list[id] == 0.0):#If no data,drawing a empty circle
                if id >= len(y_avg_log_percent_list) - predict_count and id < len(y_avg_log_percent_list):
                    #This is the data of prediction and empty
                    pl.scatter(x_ticks[id], y_avg_log_percent_list[id], c = 'r') 
                else:
                    #This is the real data of y_avg_log_percent and empty
                    pl.scatter(x_ticks[id], y_avg_log_percent_list[id], c = '', marker = 'o', edgecolors = 'r', s = 50)
            else:#Data not empty
                if id >= len(y_avg_log_percent_list) - predict_count and id < len(y_avg_log_percent_list):
                    #This is the data of prediction and not empty
                    pl.scatter(x_ticks[id], y_avg_log_percent_list[id], c = 'r') 
                    pl.text(x_ticks[id], y_avg_log_percent_list[id], '%.1f' % y_avg_log_percent_list[id], fontsize = 9)
                else:
                    #This is the real data of y_avg_log_percent and not empty
                    pl.scatter(x_ticks[id], y_avg_log_percent_list[id], c = '#60bdae') 
                    pl.text(x_ticks[id], y_avg_log_percent_list[id], '%.1f' % y_avg_log_percent_list[id], fontsize = 9)
        
        xnew_hour = []
        ynew_max_log_percent = []
        ynew_avg_log_percent = []
        
        ##In order to smooth the line chart,handle the data further##
        #Expand each x axis data 20 times
        xnew_hour = np.linspace(np.asarray(x_ticks).min(), np.asarray(x_ticks).max(), np.asarray(x_ticks).size*20) 
        #Handle the data of new y axis data
        ynew_max_log_percent = spline(np.asarray(x_ticks), np.asarray(y_max_log_percent_list), xnew_hour)
        ynew_max_log_percent_list = list(ynew_max_log_percent)
        
        ynew_avg_log_percent = spline(np.asarray(x_ticks), np.asarray(y_avg_log_percent_list), xnew_hour)
        ynew_avg_log_percent_list = list(ynew_avg_log_percent)
        #No negative value for y-axis
        for y_idx in range(len(ynew_max_log_percent_list)):
            if (ynew_max_log_percent_list[y_idx] < 0.0):
                ynew_max_log_percent_list[y_idx] = 0.0
        ynew_max_log_percent = np.asarray(ynew_max_log_percent_list)
        
        #No negative value for y-axis
        for y_idx in range(len(ynew_avg_log_percent_list)):
            if (ynew_avg_log_percent_list[y_idx] < 0.0):
                ynew_avg_log_percent_list[y_idx] = 0.0
        ynew_avg_log_percent = np.asarray(ynew_avg_log_percent_list)

        #Fill the gragh according to your requirement  tian chong yin ying
        max_ratio = len(list(predict_array_max)) / float('%.1f' % len(y_max_log_percent_list))
        pl.fill_between(xnew_hour, ynew_max_log_percent, where=(xnew_hour.min()<xnew_hour) & (xnew_hour<xnew_hour.max() * (1.0 - max_ratio)), color = '#60bdae', alpha = 0.15)
        pl.fill_between(xnew_hour, ynew_max_log_percent, where=(xnew_hour.max() * (1.0 - max_ratio) <xnew_hour) & (xnew_hour<xnew_hour.max()), color = '#60bdae', alpha = 0.5)
        #Draw curve graph
        pl.plot(xnew_hour, ynew_max_log_percent, color = '#60bdae')
        
        #Fill the gragh according to your requirement
        avg_ratio = len(list(predict_array_avg)) / float('%.1f' % len(y_avg_log_percent_list))
        pl.fill_between(xnew_hour, ynew_avg_log_percent, where=(xnew_hour.min() < xnew_hour) & (xnew_hour < xnew_hour.max() * (1.0 - avg_ratio)), color = '#4c78fb', alpha = 0.15)
        pl.fill_between(xnew_hour, ynew_avg_log_percent, where=(xnew_hour.max() * (1.0 - avg_ratio) < xnew_hour) & (xnew_hour < xnew_hour.max()), color = '#4c78fb', alpha = 0.5)
        #Draw curve graph
        pl.plot(xnew_hour, ynew_avg_log_percent, color = '#4c78fb')

        #Set the legends for the both graphs
        box = ax.get_position()
        ax.set_position([box.x0, box.y0 + box.height * 0.1, box.width, box.height * 0.9])
        ax.legend(['ACTIVE_LOG_USED_PERCENT_MAX', 'ACTIVE_LOG_USED_PERCENT_AVG'], fontsize = 9, loc = 'upper center', bbox_to_anchor=(0.5,1.06), fancybox = True, shadow = True, ncol = 4)
        
        pl.show()

In [ ]:
if(ISOK_ALL_PARA and (REPORT_TYPE == 'ALL' or REPORT_TYPE == 'LOG')):
    df_predict = model_max.predict('2018-04-23 23', '2018-04-25 15', dynamic=True)
    df_max[-1] = df_predict[0]
    fig, ax = plt.subplots(figsize=(24, 7))
    
    x_data = df_max.loc['2018-04-22 08':].index
    y_data = df_max.loc['2018-04-22 08':].values
    
    ax = df_max.loc['2018-04-22 08':].plot(ax=ax, label='ACTIVE_LOG_USED_PERCENT_MAX')
    #To mark the the value for per point
    for id in range(0,len(y_data),1):
        pl.scatter(x_data[id], y_data[id],c = '#2c628b') 
    for id in range(0,len(y_data),2):
        pl.text(x_data[id], y_data[id], '%.1f' % y_data[id], fontsize = 9)
    pl.grid(True, ls = '--', color = '#2c628b', alpha = 0.05)
    fig = model_max.plot_predict('2018-04-23 23', '2018-04-25 15', dynamic=True, ax=ax, plot_insample=False, alpha = 0.05)
    figure_title = 'Max Log Usage Percentage by Hour\n'
    pl.title(figure_title, fontsize = 14, fontweight = 'bold')
    x_lable = 'Hours'
    #To set x-axis label
    pl.xlabel(x_lable)
    pl.grid(True, ls = '--', color = '#2c628b', alpha = 0.05)
    pl.show()

In [ ]:
if(ISOK_ALL_PARA and (REPORT_TYPE == 'ALL' or REPORT_TYPE == 'LOG')):
    df_predict = model_avg.predict('2018-04-23 23', '2018-04-25 15', dynamic=True)
    df_avg[-1] = df_predict[0]
    fig, ax = plt.subplots(figsize=(24, 7))
    
    x_data = df_avg.loc['2018-04-22 08':].index
    y_data = df_avg.loc['2018-04-22 08':].values
    
    ax = df_avg.loc['2018-04-22 08':].plot(ax=ax, label='ACTIVE_LOG_USED_PERCENT_AVG')
    
    #To mark the the value for per point
    for id in range(0,len(y_data),1):
        pl.scatter(x_data[id], y_data[id],c = '#2c628b') 
    for id in range(0,len(y_data),2):
        pl.text(x_data[id], y_data[id], '%.1f' % y_data[id], fontsize = 9)
        
    pl.grid(True, ls = '--', color = '#2c628b', alpha = 0.05)
    fig = model_avg.plot_predict('2018-04-23 23', '2018-04-25 15', dynamic=True, ax=ax, plot_insample=False, alpha = 0.05)
    figure_title = 'Average Log Usage Percentage by Hour\n'
    pl.title(figure_title, fontsize = 14, fontweight = 'bold')
    pl.grid(True, ls = '--', color = '#2c628b', alpha = 0.05)
    x_lable = 'Hours'
    #To set x-axis label
    pl.xlabel(x_lable)
    pl.show()